## BOW 기반의 카운트 벡터 생성

BOW의 실습을 위해 대상 말뭉치로 NLTK가 제공하는 영화 리뷰를 사용하겠다.

In [1]:
import nltk
nltk.download('movie_reviews')

from nltk.corpus import movie_reviews

print('#review count :', len(movie_reviews.fileids())) # 영화 리뷰 문서의 id를 반환
print('#samples of file ids:', movie_reviews.fileids()[:10]) #id를 10개까지만 출력

fileid = movie_reviews.fileids()[0] # 첫번째 문서의 id를 반환
print('#id of the first review:', fileid)

# 첫번째 문서의 내용을 200자까지만 출력
print('#first review content:\n', movie_reviews.raw(fileid)[:200])

# 첫번째 문서를 sentence tokenize한 결과 중 앞 두 문장
print('#sentence tokenization result:', movie_reviews.sents(fileid)[:2])

# 첫번째 문서를 word tokenize한 결과 중 앞 20개 단어
print('#word tokenization result:', movie_reviews.words(fileid)[:20])

#review count : 2000
#samples of file ids: ['neg/cv000_29416.txt', 'neg/cv001_19502.txt', 'neg/cv002_17424.txt', 'neg/cv003_12683.txt', 'neg/cv004_12641.txt', 'neg/cv005_29357.txt', 'neg/cv006_17022.txt', 'neg/cv007_4992.txt', 'neg/cv008_29326.txt', 'neg/cv009_29417.txt']
#id of the first review: neg/cv000_29416.txt
#first review content:
 plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . 
w
#sentence tokenization result: [['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.'], ['they', 'get', 'into', 'an', 'accident', '.']]
#word tokenization result: ['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.', 'they', 'get', 'into', 'an']


[nltk_data] Downloading package movie_reviews to
[nltk_data]     /data/ydkim/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


먼저 각 문서에 대한 토큰화 결과들로 리스트를 만들기 위해 아래 코드를 실행한다.  
fileids()를 이용해 모든 문서의 id를 가져오고 각 id들에 대해 words()로 토큰화 결과를 가져와 리스트를 만든다.

In [2]:
documents = [list(movie_reviews.words(fileid)) for fileid in movie_reviews.fileids()]
print(documents[0][:50]) # 첫번째 문서의 앞 50개 단어 출력

['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.', 'they', 'get', 'into', 'an', 'accident', '.', 'one', 'of', 'the', 'guys', 'dies', ',', 'but', 'his', 'girlfriend', 'continues', 'to', 'see', 'him', 'in', 'her', 'life', ',', 'and', 'has', 'nightmares', '.', 'what', "'", 's', 'the', 'deal', '?', 'watch']


이제 특성 집합을 만들기 위해 딕셔너리를 써서 단어별로 말뭉치 전체에서의 빈도를 계산하고, 빈도가 높은 단어부터 정렬한 후에 빈도수 상위 20개의 단어를 출력한다. 

In [3]:
word_count = {}
for text in documents:
    for word in text:
        word_count[word] = word_count.get(word, 0) + 1
        
sorted_features = sorted(word_count, key=word_count.get, reverse=True)
for word in sorted_features[:10]:
    print(f"count of '{word}': {word_count[word]}", end=', ')

count of ',': 77717, count of 'the': 76529, count of '.': 65876, count of 'a': 38106, count of 'and': 35576, count of 'of': 34123, count of 'to': 31937, count of ''': 30585, count of 'is': 25195, count of 'in': 21822, 

위 결과를 보면 ',', 'the', 'a'와 같이 의미적으로 쓸모없는 단어만 빈도가 높은 것 같아 보인다. 이를 해결하기 위해 정규표현식으로 다시 토큰화하자.  
이를 위해 먼저 raw()를 이용해 원문을 가져와서 documents를 만들고, 이에 대해 토큰화를 한다. 이왕 하는 김에 NLTK가 제공하는 불용어 사전을 이용해 불용어도 제거한다.

In [4]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

tokenizer = RegexpTokenizer("[\w']{3,}") # 정규표현식으로 토크나이저를 정의
english_stops = set(stopwords.words('english')) # 영어 불용어를 가져옴

# words() 대신 raw()로 원문을 가져옴
documents = [movie_reviews.raw(fileid)]

# stopwords의 적용과 토큰화를 동시에 수행.
tokens = [[token for token in tokenizer.tokenize(doc) if token not in english_stops] for doc in documents]

word_count = {}
for text in documents:
    for word in text:
        word_count[word] = word_count.get(word, 0) + 1
        
sorted_features = sorted(word_count, key=word_count.get, reverse=True)

print('num of features:', len(sorted_features))
for word in sorted_features[:10]:
    print(f"count of '{word}': {word_count[word]}", end=', ')

num of features: 48
count of ' ': 825, count of 'e': 382, count of 't': 327, count of 'i': 234, count of 'a': 224, count of 'o': 216, count of 'n': 195, count of 's': 192, count of 'h': 172, count of 'r': 166, 

상위 빈도수를 가지는 단어 천 개만 추출해서 최종적으로 문서를 표현할 특성으로 사용하기로 한다.이때 중요한 것은, 특성 집합에는 단어의 순서가 있으며 이 순서에 따라 문서의 카운트 벡터 값이 결정된다는 것이다.

In [5]:
# 빈도가 높은 상위 1000개의 단어만 추출해 features를 구성
word_features = sorted_features[:1000]

이제 주어진 문서를 특성 벡터, 즉 카운트 벡터로 변환하는 함수를 만든다.

함수가 제대로 작동하는지 알아보기 위해 아래와 같이 입, 출력의 예를 만들어 테스트를 진행한다.

In [6]:
# 주어진 document를 feature로 변환하는 함수, word_features를 사용
def document_features(document, word_features):
    word_count = {}
    for word in document: # document에 있는 단어들에 대해 빈도수를 먼저 계산
        word_count[word] = word_count.get(word, 0) + 1
        
    features = []
    # word_features의 단어에 대해 계산된 빈도수를 features에 추가
    for word in word_features:
        features.append(word_count.get(word, 0)) # 빈도가 없는 단어는 0을 입력
    return features

In [7]:
word_features_ex = ['one', 'two', 'teen', 'couples', 'solo']
doc_ex = ['two', 'two', 'couples']

print(document_features(doc_ex, word_features_ex))

[0, 2, 0, 1, 0]


In [8]:
feature_set = [document_features(d, word_features) for d in tokens]

# 첫째 feature set의 내용을 앞 20개만 word_features의 단어와 함께 출력
for i in range(20):
    print(f'({word_features[i]}, {feature_set[0][i]})', end=', ')

( , 0), (e, 0), (t, 0), (i, 0), (a, 0), (o, 0), (n, 0), (s, 0), (h, 0), (r, 0), (l, 0), (d, 0), (m, 0), (u, 0), (g, 0), (c, 0), (w, 0), (f, 0), (y, 0), (p, 0), 

## 사이킷런으로 카운트 벡터 생성

사이킷런은 자체적인 토크나이저를 지원하므로, 사용자가 별도로 미리 토큰화를 하지 않아도 된다.  
그러나 좀 더 세부적인 조정을 통해 성능을 높이고 싶을 때는 토크나이저를 함수로 정의하고 사이킷런에서 이를 사용하게 할 수 있다.  
한글의 경우에는 KoNLPy로 형태소 분석을 수행해야 하므로 반드시 별도의 토크나이저를 사용해야 한다.

이 절에서는 위에서 사용한 movie_reviews에 대해 사이킷런의 CountVector 클래스를 이용해 카운트 벡터를 생성한다.

In [9]:
# data 준비, movie_reviews.raw()를 사용해 raw text를 추출
reviews = [movie_reviews.raw(fileld) for fileld in movie_reviews.fileids()]

CountVectorizer 객체를 생성할 때 vocabulary 매개변수를 쓰면 앞에서 만든 word_features에 있는 단어들만으로 벡터를 구성할 수 있다.  

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

# cv = CountVectorizer() # 모든 매개변수에 기본값을 사용하는 경우

# 앞에서 생성한 word_features로 특성 집합을 지정하는 경우
cv = CountVectorizer(vocabulary=word_features)

# 특성 집합을 지정하지 않고 최대 특성의 수를 지정하는 경우
# cv = CountVectorizer(max_feautures=1000)

print(cv) # 객체의 인수를 확인

CountVectorizer(vocabulary=[' ', 'e', 't', 'i', 'a', 'o', 'n', 's', 'h', 'r',
                            'l', 'd', 'm', 'u', 'g', 'c', 'w', 'f', 'y', 'p',
                            ',', 'b', 'v', '\n', '.', 'k', "'", '0', '"', '-', ...])


객체를 생성했다면 아래와 같이 fit_transform()으로 특성 집합을 생성하고 카운트 벡터를 생성한다.  
get_feature_names_out()를 출력하면 word_features와 사용된 단어 및 순서가 동일한 것을 볼 수 있다.

In [11]:
reviews_cv = cv.fit_transform(reviews) # reviews를 이용해 count vector를 학습하고, 변환
print(cv.get_feature_names_out()[:20]) # count vector에 사용된 feature 이름을 반환
print(word_features[:20]) # 비교를 위해 출력

[' ' 'e' 't' 'i' 'a' 'o' 'n' 's' 'h' 'r' 'l' 'd' 'm' 'u' 'g' 'c' 'w' 'f'
 'y' 'p']
[' ', 'e', 't', 'i', 'a', 'o', 'n', 's', 'h', 'r', 'l', 'd', 'm', 'u', 'g', 'c', 'w', 'f', 'y', 'p']


## 한국어 텍스트의 카운트 벡터 반환

In [12]:
import pandas as pd
df = pd.read_csv('./data/daum_movie_review.csv')
df.head()

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워


여기서는 review 항목으로부터 카운트 벡터를 생성하는 것에 집중한다.

먼저 영어 문서분류와 동일하게 자체 토크나이저를 사용해서 카운트 벡터를 생성한다.

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
daum_cv = CountVectorizer(max_features=1000)

# review를 이용해 count vector를 학습하고 반환
daum_DTM = daum_cv.fit_transform(df.review)

print(daum_cv.get_feature_names_out()[:100])

['10점' '18' '1987' '1도' '1점' '1점도' '2시간' '2시간이' '2편' '5점' '6점' '7점' '8점'
 'cg' 'cg가' 'cg는' 'cg도' 'cg만' 'good' 'of' 'ㅋㅋ' 'ㅋㅋㅋ' 'ㅋㅋㅋㅋ' 'ㅎㅎ' 'ㅎㅎㅎ'
 'ㅜㅜ' 'ㅠㅠ' 'ㅠㅠㅠ' 'ㅡㅡ' '가는' '가는줄' '가면' '가서' '가슴' '가슴아픈' '가슴이' '가장' '가족'
 '가족과' '가족들과' '가족의' '가족이' '가지고' '간만에' '갈수록' '감독' '감독님' '감독은' '감독의' '감독이'
 '감동' '감동과' '감동도' '감동은' '감동을' '감동이' '감동입니다' '감동적' '감동적이고' '감동적인' '감사드립니다'
 '감사합니다' '감정이' '갑자기' '갔는데' '갔다가' '강철비' '강추' '강추합니다' '같고' '같네요' '같다' '같습니다'
 '같아' '같아요' '같은' '같은데' '같음' '같이' '개연성' '개연성이' '개인적으로' '거의' '겁나' '것도' '것은'
 '것을' '것이' '것이다' '겨울왕국' '결국' '결말' '결말이' '계속' '고맙습니다' '곤지암' '공포' '공포를'
 '공포영화' '관객']


위 결과는 문제가 많아보인다. 'cg'가 들어간 단어들이 전부 별도의 단어로 분류됐으며, '감동'이 들어간 단어는 더 심하다. 역시 KoNLPy의 형태소 분석기를 사용하자.

In [14]:
from konlpy.tag import Okt
twitter_tag = Okt()

print('#전체 형태소 결과:', twitter_tag.morphs(df.review[1]))
print('#명사만 추출:', twitter_tag.nouns(df.review[1]))
print('#품사 태깅 결과:', twitter_tag.pos(df.review[1]))

#전체 형태소 결과: ['몰입', '할수밖에', '없다', '.', '어렵게', '생각', '할', '필요없다', '.', '내', '가', '전투', '에', '참여', '한', '듯', '손', '에', '땀', '이남', '.']
#명사만 추출: ['몰입', '생각', '내', '전투', '참여', '듯', '손', '땀', '이남']
#품사 태깅 결과: [('몰입', 'Noun'), ('할수밖에', 'Verb'), ('없다', 'Adjective'), ('.', 'Punctuation'), ('어렵게', 'Adjective'), ('생각', 'Noun'), ('할', 'Verb'), ('필요없다', 'Adjective'), ('.', 'Punctuation'), ('내', 'Noun'), ('가', 'Josa'), ('전투', 'Noun'), ('에', 'Josa'), ('참여', 'Noun'), ('한', 'Determiner'), ('듯', 'Noun'), ('손', 'Noun'), ('에', 'Josa'), ('땀', 'Noun'), ('이남', 'Noun'), ('.', 'Punctuation')]


위 결과를 보면 전체 형태소를 다 사용하는 경우에는 필요없는 단어가 많아보이고, 명사만 쓰면 또 너무 적어보인다.  
여기서는 명사, 동사, 형용사 세 개를 선택하기로 한다.

In [15]:
def my_tokenizer(doc):
    return [
        token
        for token, pos in twitter_tag.pos(doc)
        if pos in ['Noun', 'Verb', 'Adjective']
    ]

print('나만의 토크나이저 결과:', my_tokenizer(df.review[1]))

나만의 토크나이저 결과: ['몰입', '할수밖에', '없다', '어렵게', '생각', '할', '필요없다', '내', '전투', '참여', '듯', '손', '땀', '이남']


아래와 같이 tokenizer 매개변수에 my_tokenizer 함수를 지정해 객체를 생성한 후에, fit_transform으로 카운트 벡터를 생성한다.

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

# 토크나이저와 특성의 최대 개수를 지정
# 명사만 추출하고 싶은 경우에는 tokenizer에 'twitter_tag.nouns'를 바로 지정해도 됨
daum_cv = CountVectorizer(max_features=1000, tokenizer=my_tokenizer)

# review를 이용해 count vector를 학습하고, 변환
daum_DTM = daum_cv.fit_transform(df.review)

print(daum_cv.get_feature_names_out()[:100]) # count vector에 사용된 feature 이름을 반환

['가' '가는' '가는줄' '가면' '가서' '가슴' '가장' '가족' '가족영화' '가지' '가치' '각색' '간' '간다'
 '간만' '갈' '갈수록' '감' '감독' '감동' '감사' '감사합니다' '감상' '감성' '감정' '감탄' '갑자기' '갔는데'
 '갔다' '갔다가' '강' '강철' '강추' '같고' '같네요' '같다' '같습니다' '같아' '같아요' '같은' '같은데'
 '같음' '개' '개그' '개봉' '개연' '개인' '거' '거기' '거리' '거의' '걱정' '건' '건가' '건지' '걸'
 '겁니다' '것' '게' '겨울왕국' '결론' '결말' '경찰' '경험' '계속' '고' '고맙습니다' '고민' '고생' '곤지암'
 '곳' '공감' '공포' '공포영화' '과' '과거' '관' '관객' '관객수' '관람' '광주' '괜찮은' '교훈' '구성'
 '국내' '국민' '군인' '군함도' '굿' '권선' '귀신' '그' '그것' '그게' '그날' '그냥' '그닥' '그대로'
 '그때' '그래픽']


## 카운트 벡터의 활용

다음 예에서는 NLTK 영화 리뷰 문서들에 대해 유사도를 측정해본다.  
제대로 유사도가 계산되는지 확인하기 위해, 먼저 유사도를 측정할 대상 문서를 만든다. 아래 예에서는 첫째 리뷰의 뒷부분 절반을 잘라서 대상 문서를 만들었다.

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

# 첫째 리뷰의 문자수를 확인하고 뒤 절반을 가져오기 위해 중심점을 찾음
start = len(reviews[0]) // 2

# 중심점으로부터 뒤 절반을 가져와서 비교할 문서를 생성
source = reviews[0][-start:]

# 코사인 유사도는 카운트 벡터에 대해 계산하므로 벡터로 변환
# transform은 반드시 리스트나 행렬 형태의 입력을 요구하므로 리스트로 만들어서 입력
source_cv = cv.transform([source])

# 행렬의 크기를 확인, 문서가 하나이므로 (1, 1000)
print('#대상 특성 행렬의 크기:', source_cv.shape)

# 변환된 count vector와 기존 값들과의 similarity 계산
sim_result = cosine_similarity(source_cv, reviews_cv)

print('#유사도 계산 행렬의 크기:', sim_result.shape)
print('#유사도 계산결과를 역순으로 정렬:', sorted(sim_result[0], reverse=True)[:10])

#대상 특성 행렬의 크기: (1, 48)
#유사도 계산 행렬의 크기: (1, 2000)
#유사도 계산결과를 역순으로 정렬: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
